<a href="https://colab.research.google.com/github/DSKI23/alzheimervalentin/blob/13-logged-hyperparameters/LoggedHyperparameters/logged-hyperparameters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
!pip install -r "https://raw.githubusercontent.com/DSKI23/alzheimervalentin/refs/heads/main/requirements_colab.txt"

DatasetDict({
    train: Dataset({
        features: ['Age', 'BMI', 'Education Level', 'Gender', 'Family History of Alzheimer’s', 'Cognitive Test Score', 'Genetic Risk Factor (APOE-ε4 allele)', 'Alzheimer’s Diagnosis'],
        num_rows: 59426
    })
    validation: Dataset({
        features: ['Age', 'BMI', 'Education Level', 'Gender', 'Family History of Alzheimer’s', 'Cognitive Test Score', 'Genetic Risk Factor (APOE-ε4 allele)', 'Alzheimer’s Diagnosis'],
        num_rows: 14857
    })
})

In [ ]:
from huggingface_hub import notebook_login
from datasets import load_dataset
import wandb
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np
import pickle
import os

In [ ]:

notebook_login()

In [29]:
!wandb login

wandb: Currently logged in as: zhannalialko (zhannalialko-dhbw-mosbach) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [31]:
df.columns

Index(['Age', 'BMI', 'Education Level', 'Gender',
       'Family History of Alzheimer’s', 'Cognitive Test Score',
       'Genetic Risk Factor (APOE-ε4 allele)', 'Alzheimer’s Diagnosis'],
      dtype='object')

In [42]:

dataset = load_dataset("DS23-KI-Projekt/alzheimerdataset_split")
dataset

# Function to load dataset
def load_data():
    try:
        df = dataset["train"].to_pandas()
        target = "Alzheimer’s Diagnosis"
        features = [col for col in df.columns if col != target]
        return df[features], df[target]
    except Exception as e:
        print(f"Error loading dataset: {e}")
        exit(1)

# Function to train and evaluate Random Forest
def train_rf(X_train, X_test, y_train, y_test, n_estimators):
    rf_model = RandomForestClassifier(n_estimators=n_estimators, random_state=42)
    rf_model.fit(X_train, y_train)

    y_pred = rf_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    class_report = classification_report(y_test, y_pred, output_dict=True)  # JSON format for WandB
    cfm = confusion_matrix(y_test, y_pred).tolist()  # Convert to list for WandB logging

    return rf_model, accuracy, class_report, cfm

#  Main function
def main():
    # Initialize WandB
    wandb.init(entity="zhannalialko-dhbw-mosbach", project="alzheimer-rf", name="RF-Training")

    # Load dataset
    X, y = load_data()

    # Split dataset (80% train, 20% test)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

    best_model = None
    best_accuracy = 0

    #  Train over 10 epochs with increasing estimators
    for epoch in range(1, 11):
        n_estimators = epoch * 10  # Start with 10 trees, increase to 100

        rf_model, accuracy, class_report, cfm = train_rf(X_train, X_test, y_train, y_test, n_estimators)

        # Save best model
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model = rf_model

        # Log results in WandB
        wandb.log({
            "epoch": epoch,
            "accuracy": accuracy,
            "n_estimators": n_estimators,
            "classification_report": class_report,
            "confusion_matrix": cfm
        })

        print(f"Epoch {epoch} | n_estimators={n_estimators} | Accuracy={accuracy:.4f}")

    #  Save the best model
    model_dir = "random_forest_alzheimer"
    os.makedirs(model_dir, exist_ok=True)

    with open(f"{model_dir}/best_rf_model.pkl", "wb") as file:
        pickle.dump(best_model, file)

    print(f"Best model saved with accuracy: {best_accuracy:.4f}")

    # Finish WandB logging
    wandb.finish()

#  Run the script
if __name__ == "__main__":
    main()

Epoch 1 | n_estimators=10 | Accuracy=0.6820
Epoch 2 | n_estimators=20 | Accuracy=0.6946
Epoch 3 | n_estimators=30 | Accuracy=0.6959
Epoch 4 | n_estimators=40 | Accuracy=0.6977
Epoch 5 | n_estimators=50 | Accuracy=0.6974
Epoch 6 | n_estimators=60 | Accuracy=0.6980
Epoch 7 | n_estimators=70 | Accuracy=0.6990
Epoch 8 | n_estimators=80 | Accuracy=0.6986
Epoch 9 | n_estimators=90 | Accuracy=0.6987
Epoch 10 | n_estimators=100 | Accuracy=0.6987
Best model saved with accuracy: 0.6990


accuracy,▁▆▇▇▇█████
epoch,▁▂▃▃▄▅▆▆▇█
n_estimators,▁▂▃▃▄▅▆▆▇█
accuracy,0.69872
epoch,10
n_estimators,100
